In [38]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests
import pymongo

In [39]:
executable_path = {'executable_path': r'/Users/crobinson1205/Desktop/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

mongo_export = {}

In [40]:
#view website
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
website  = browser.html



In [41]:
#get title and content from website
article_soup = BeautifulSoup(website, 'html.parser')


content = article_soup.find('div',class_ = 'react_grid_list').find('div').find('ul').find('li').find('div').find(class_ = 'rollover_description_inner').text


title = article_soup.find('div',class_ = 'react_grid_list').find('div').find('ul').find('li').find('div').find('a').find(class_ = 'bottom_gradient').text




article_soup_dict = {}
article_soup_dict["title"] = title
article_soup_dict["content"] = content
article_soup_dict

{'title': "Mars InSight Lander to Push on Top of the 'Mole'",
 'content': 'Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.'}

#### 

In [42]:
#get image from jpl space
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
website = browser.html

In [43]:
image_soup = BeautifulSoup(website, 'html.parser')
main_section = image_soup.find(class_="centered_text clearfix main_feature primary_media_feature single")
image_con = main_section.find(class_ ="default floating_text_area ms-layer")
image_str = image_con.find('a').get('data-fancybox-href')
image_link = f'https://www.jpl.nasa.gov{image_str}'

image_soup_dict = {}
image_soup_dict["link"] = image_link
image_soup_dict

{'link': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA01322_ip.jpg'}

In [44]:
#mars twitter weather
url = 'https://twitter.com/marswxreport?lang=en/tweets'
data = requests.get(url) 
twitter_soup = BeautifulSoup(data.text, 'html.parser')

In [45]:
mars_weather_tweet = twitter_soup.find_all('div', attrs={"class": "tweet"})[0]
mars_weather_tweet.find('p').text

found = False
i = 0
while found == False:
    mars_weather_tweet = twitter_soup.find_all('div', attrs={"class": "tweet"})[i]
    mars_weather_text = mars_weather_tweet.find('p').text
    if (mars_weather_text.find('InSight sol') != -1) and (mars_weather_text.find('ºC') != -1) and (mars_weather_text.find('ºF') != -1) and (mars_weather_text.find('m/s') != -1):
        weather_report = mars_weather_tweet.find('p').text
        found = True
    
    i+=1
    
weather_soup_dict = {}
weather_soup_dict["report"] = weather_report
weather_soup_dict

{'report': 'InSight sol 441 (2020-02-22) low -95.1ºC (-139.1ºF) high -13.5ºC (7.7ºF)\nwinds from the SSE at 6.2 m/s (13.8 mph) gusting to 21.0 m/s (47.0 mph)\npressure at 6.30 hPapic.twitter.com/ztf0TqK14M'}

In [46]:
url = 'https://space-facts.com/mars/'
browser.visit(url)
website = browser.html

fact_soup = BeautifulSoup(website, 'html.parser')
mars_facts = fact_soup.find('table',id="tablepress-p-mars")

mars_facts_rows = mars_facts.find_all('tr')
mars_table = {}

i = 0
for i in range(len(mars_facts_rows)):
    mars_elements = []
    key = mars_facts_rows[i].find('td',class_ = 'column-1').text
    value = mars_facts_rows[i].find('td',class_ = 'column-2').text
    mars_table[key]=[value]

mars_df = pd.DataFrame(mars_table)
mars_df = mars_df.transpose()

mars_df = mars_df.rename_axis('Description')
mars_df = mars_df.rename(columns = {0:'Value'})

# mars_df
mars_df.to_html()

fact_soup_dict = {}
fact_soup_dict["facts"] = mars_df.to_html()
fact_soup_dict

{'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n

In [47]:
#mars hemispheres pictures
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

website = browser.html

hemisphere_soup = BeautifulSoup(website, 'html.parser')
hemisphere_soup.select('.itemLink.product-item')[0]

<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a>

In [48]:
all_mars_pics = []

j = 0

i = 0

while i < len(hemisphere_soup.select('.itemLink.product-item h3')):
    
    hemisphere_soup = BeautifulSoup(website, 'html.parser')
    
    pics_link = hemisphere_soup.select('.itemLink.product-item')[j].get('href')
    
    destination = f'https://astrogeology.usgs.gov{pics_link}'
    
    planet_pics = {}
    
    browser.visit(destination)
    
    destinaton_site = browser.html
    
    destination_soup = BeautifulSoup(destinaton_site,'html.parser')
    
    final_pic = browser.links.find_by_text('Sample')
    
    planet_pics['url'] = final_pic['href']

    all_mars_pics.append(planet_pics)
    
    j += 2
    
    i += 1

all_mars_pics

hemisphere_soup_list = []
hemisphere_soup_list.append(all_mars_pics)
hemisphere_soup_list


[[{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]]

In [49]:
mongo_export = {}
mongo_export['article_soup'] = article_soup_dict
mongo_export['image_soup'] = image_soup_dict
mongo_export['fact_soup'] = fact_soup_dict
mongo_export['hemisphere_soup'] = hemisphere_soup_list
mongo_export['weather_soup'] = weather_soup_dict
mongo_export


{'article_soup': {'title': "Mars InSight Lander to Push on Top of the 'Mole'",
  'content': 'Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.'},
 'image_soup': {'link': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA01322_ip.jpg'},
 'fact_soup': {'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n   

In [50]:
browser.quit()

In [51]:
#pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [52]:
db = client.marsDB

mars = db.mars_info

mars.update({},mongo_export, upsert = True)


/Users/crobinson1205/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}